# 자연어와 단어의 분산 표현

## 통계 기반 기법
파이썬으로 말뭉치 처리하기


In [1]:
text = 'You say goodbye and I say hello.'
test = text.lower()
text = text.replace('.', ' .')
print(text)

You say goodbye and I say hello .


In [3]:
words = text.split(' ')
print(words)

['You', 'say', 'goodbye', 'and', 'I', 'say', 'hello', '.']


In [4]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [5]:
print(id_to_word)
print(word_to_id)

{0: 'You', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'I', 5: 'hello', 6: '.'}
{'You': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'I': 4, 'hello': 5, '.': 6}


In [6]:
print(id_to_word[1])
print(word_to_id['hello'])

say
5


In [7]:
import numpy as np

corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
print(corpus)

[0 1 2 3 4 1 5 6]


In [9]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [10]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)